# Merchant Rating and Ranking

In [1]:
import pandas as pd

In [11]:
import ray

ray.shutdown()

In [ ]:
ray.init()

In [2]:
all_order_train = pd.read_pickle('../all_order_train.pkl')
print(all_order_train.shape)

all_order_train.head()

(33720820, 12)


,order_id,user_id,order_number,order_hour_of_day,product_id,purchase_date,merchant,product_name,price,aisle,department,week_number
0,2539329,1,1,8,196,2019-02-13,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,7
1,2231262,31,17,11,196,2019-03-10,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
2,3058369,195,34,10,196,2019-02-25,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,9
3,2257155,951,3,20,196,2019-03-09,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10
4,1121647,992,7,10,196,2019-03-08,L&M Markets (Hometown Grocers Co-op),Soda,3.08,soft drinks,beverages,10


## Calculate Rating Score

* `customer_loyalty = returned customers this period / total customers this period`, [0,1] range
* `sales_performance = # of orders above past average sales this period/ # of orders this period`, [0,1] range
* `rating = 1 - sqrt((power(1-customer_loyalty, 2) + (power(1-sales_performance, 2))/2)`
  * It uses the distance between perfect score 1 and current point

In [ ]:
def get_customer_loyalty(current_week)

In [10]:
test_merchant = '49th Parallel Grocery'